# "Universal" connection and data base manager

In [ ]:
# Class of connection to data bases (modules must have standart class "connect")

import pymssql

class Connection:
    """
    Universal connection to SQL.
    """
    
    def __init__(self, db):
        """
        Initialization.
        
        Parameters:
            db["db_name"] (str) - database name (oracle/ms_server/teradata/sqlite)
            db["db_config"] (dict) - database connection configuration
        """
        
        self.db_name = db["db_name"]
        self.db_config = db["db_config"]
        
        
    def _database_choice(self):
        """
        Choose module for connection.
        """
        
        if self.db_name == "oracle":
            return cx_Oracle
        elif self.db_name == "ms_server":
            return pymssql
        elif self.db_name == "sqlite":
            return sqlite3
        elif self.db_name == "teradata":
            return teradatasql
        else:
            raise ValueError("Failed database name")

    
    def __enter__(self):
        """
        Create connection.
        """
        
        self.connection = self._database_choice().connect(**self.db_config)
        
        return self.connection
    
    
    def __exit__(self, *args):
        """
        Close connection.
        """
        
        self.connection.close()

In [48]:
# Class of data base manager (data base on standart SQL dialect)

import pandas as pd

class DataBaseManager:
    """
    Manage data base on SQL.
    """
    
    def __init__(self, db):
        """
        Initialization.
        
        Parameters:
            db["db_name"] (str) - database name (oracle/ms_server/teradata/sqlite)
            db["db_config"] (dict) - database connection configuration
        """
        
        self.db = db
        
        
    def test_connection(self):
        with Connection(self.db) as conn:
            cur = conn.cursor()
            conn.commit()
    
    
    def select(self, request, pd_mode=False, **params):
        """
        Load data from base.
        """
        
        with Connection(self.db) as conn:
            if pd_mode:
                data = pd.read_sql_query(sql=request, con=conn)
                if params.get("columns", False):
                    data.columns = params["columns"]
                if params.get("dtype", False):
                    for col, type_ in params["dtype"]:
                        data[col] = data[col].astype(type_)
            else:
                cur = conn.cursor()
                cur.execute(request)
                data = cur.fetchall()
            conn.commit()
        
        return data
    
    
    def insert(self, request, data, step=None):
        """
        Upload data to base.
        """
        
        cond = isinstance(data, pd.DataFrame)
        
        with Connection(self.db) as conn:
            cur = conn.cursor()
            if step:
                for ind in range(0, len(data), step):
                    batch = data.iloc[ind:ind+step].values if cond else data.iloc[ind:ind+step]
                    cur.executemany(request, batch)
            else:
                if cond:
                    data = data.values
                cur.executemany(request, data)
            conn.commit()
    
    
    def inner_action(self, request):
        """
        Inner action in base.
        """
        
        with Connection(self.db) as conn:
            cur = conn.cursor()
            cur.execute(request)
            conn.commit()
            
        
    def drop_table(self, table_name):
        """
        Drop table.
        
        Parameters:
            table_name (str) - table name
        """
        
        self.inner_action(f"DROP TABLE {table_name}")
        
        
    def truncate_table(self, table_name):
        """
        Truncate table.
        
        Drop table.
        
        Parameters:
            table_name (str) - table name
        """
        
        self.inner_action(f"TRUNCATE TABLE {table_name}")
        
        
    def is_table(self, table_name):
        """
        Test to table exist.
        
        Drop table.
        
        Parameters:
            table_name (str) - table name
            
        Returns:
            verdict (bool) - True if table exist else False
        """
        
        try:
            self.inner_action(f"SELECT 1 FROM {table_name}")
            return True
        except:
            return False